## Import packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from sklearn.preprocessing import OneHotEncoder

import pickle

%matplotlib inline

## Read data

In [2]:
train_df = pd.read_excel(f'dataset_splits/train_datset.xlsx')
eval_df = pd.read_excel(f'dataset_splits/test_datset.xlsx')

In [3]:
def innovate_transform(transform_df: pd.DataFrame, train_df: pd.DataFrame):

    null_columns_names = ['airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'buildingclasstypeid', 'buildingqualitytypeid',
       'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13',
       'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fireplacecnt', 'hashottuborspa',
       'heatingorsystemtypeid', 'poolcnt', 'poolsizesum', 'pooltypeid10',
       'pooltypeid2', 'pooltypeid7', 'propertyzoningdesc',
       'regionidneighborhood', 'storytypeid', 'threequarterbathnbr',
       'typeconstructiontypeid', 'unitcnt', 'yardbuildingsqft17',
       'yardbuildingsqft26', 'fireplaceflag', 'taxdelinquencyflag',
       'taxdelinquencyyear']

    transform_df.drop(columns = null_columns_names, inplace=True)

    dropped_columns_names = ['Unnamed: 0', 'fips', 'regionidcounty', 'structuretaxvaluedollarcnt',
                        'landtaxvaluedollarcnt', 'taxamount', 'parcelid', 'finishedsquarefeet12',
                        'rawcensustractandblock'
                        ]

    transform_df.drop(columns = dropped_columns_names, inplace=True)

    transform_df['garagetotalsqft'].fillna(train_df['garagetotalsqft'].median(), inplace=True)
    transform_df['numberofstories'].fillna(train_df['numberofstories'].mode()[0], inplace=True)
    transform_df['regionidcity'].fillna(train_df['regionidcity'].mode()[0], inplace=True)
    transform_df['garagecarcnt'].fillna(train_df['garagecarcnt'].mode()[0], inplace=True)

    transform_df = transform_df.dropna(subset=['regionidzip']).reset_index(drop=True)

    need_encoded_columns = ['propertycountylandusecode',
                         'propertylandusetypeid', 
                         'regionidcity', 'regionidzip', 'censustractandblock']

    
    with open('one-hot-encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)

    column_names = encoder.get_feature_names(need_encoded_columns)

    encoded_features = pd.DataFrame(encoder.transform(transform_df[need_encoded_columns]).toarray(), columns = column_names)

    transform_df.drop(columns = need_encoded_columns, inplace=True)

    processed_df = pd.concat([transform_df, encoded_features], axis = 1)

    return processed_df

In [4]:
processed_df = innovate_transform(eval_df, train_df)

/Users/SPeruri/miniforge3/envs/tf_eebbc_uk/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 169 entries, pictureid to censustractandblock_60591100000000
dtypes: float64(158), int64(11)
memory usage: 6.4 MB
